In [ ]:
import math

#Function to calculate distance between two cordinates
def retrieveDistanceBetweenCordinates(stationCode, lat1, long1, lat2, long2):
    degreeToRadian = math.pi / 180.0
    latDiff = (lat2 - lat1) * degreeToRadian
    longDiff = (long2 - long1) * degreeToRadian
    a = math.pow(math.sin((latDiff / 2.0)), 2) + math.cos(lat1 * degreeToRadian) * math.cos(lat2 * degreeToRadian) * math.pow((math.sin(longDiff/2.0)), 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = 6367 * c
    
    return pd.Series([stationCode, d])

In [ ]:
#Read pollutant release data-sets

os.chdir('E:/Research/Datasets/Pollutant emissions data/E-PRTR_database_v16_csv')
pollutant_release = pd.read_csv('dbo.PUBLISH_POLLUTANTRELEASE.csv')
pollutant_release_report_transfer = pd.read_csv('dbo.PUBLISH_POLLUTANTRELEASEANDTRANSFERREPORT.csv')
pollutant_release_report = pd.read_csv('dbo.PUBLISH_FACILITYREPORT.csv', encoding = "ISO-8859-1")
#Merge polutant release report dataset with polutant release and transfer dataset to add reporting year
pollutant_release_report =  pd.merge(pollutant_release_report[['FacilityReportID','PollutantReleaseAndTransferReportID', 'FacilityID']], pollutant_release_report_transfer[['PollutantReleaseAndTransferReportID', 'ReportingYear']], on='PollutantReleaseAndTransferReportID')
#Merge the above dataset with pollutant release dataset to populate repoting year and facility ids
pollutant_release = pd.merge(pollutant_release, pollutant_release_report[['FacilityReportID', 'FacilityID', 'ReportingYear']], on='FacilityReportID')
#Drop unneccessary columns
pollutant_release.drop(['ConfidentialIndicator', 'ConfidentialityReasonCode', 'ConfidentialityReasonName'], inplace=True, axis=1)
#Filter out data reported before 2012 and non AIR pollutant release
pollutant_release = pollutant_release.loc[(pollutant_release['ReportingYear'] >= 2013) & (pollutant_release['ReleaseMediumCode'] == 'AIR')]

#Filter out duplicates and countries not present in air quality dataset
reporting_facilities = facilities[['FacilityID', 'Lat', 'Long', 'CountryName']].drop_duplicates('FacilityID')
reporting_facilities = reporting_facilities.loc[reporting_facilities["CountryName"].isin(airQualityStationsMeanReadings["CountryName"].unique())]
reporting_facilities = reporting_facilities.loc[reporting_facilities['FacilityID'].isin(pollutant_release['FacilityID'].unique())]

In [ ]:
#script to polpulate nearby air quality station to facility for each pollutant 
import time
start = time.process_time()
locationMergeDataset = []
years = [2017, 2013, 2014, 2015, 2016]
polutants = ["SO2", "C6H6", "CO", "PM10", "PM2.5", "O3"]
#Function to extract nearest station reporting air quality for pollutant release facility
def extractNearestStation(row):
    #filter country wise to limit all data processing
    country_stations = airQualityStationsMeanReadings[airQualityStationsMeanReadings['CountryName'] == row['CountryName']]
    if country_stations.shape[0] > 0:
        for year in years:
            yearRecord = {}
            yearRecord['Year'] = year
            yearRecord['FacilityID'] = row['FacilityID']
            yearRecord['CountryName'] = row['CountryName']
            #Filter for processing year
            allPolutantStations = country_stations.loc[country_stations['Year'] == year]
            #Find distance from all air quality stations
            distanceMatrix = allPolutantStations.apply(lambda x: retrieveDistanceBetweenCordinates(x['AirQualityStationEoICode'], row['Lat'], row['Long'], x['Latitude'], x['Longitude']), axis=1)
            distanceMatrix.rename(columns={0:'NearestStationEoICode', 1:'Distance'},inplace=True)
            for polutant in polutants:
                requiredStations = allPolutantStations.loc[allPolutantStations[polutant].notna()]
                if requiredStations.shape[0] > 0:
                    #Find the stations which have concentration of pollutant for the processing year populated
                    requiredStationsDistance = distanceMatrix.loc[distanceMatrix['NearestStationEoICode'].isin(requiredStations["AirQualityStationEoICode"].unique())]
                    #Store the one with minimum distance
                    nearest_station = requiredStationsDistance.loc[(requiredStationsDistance['Distance'].idxmin())]
                    yearRecord[polutant] = nearest_station[0]
                    yearRecord["Dist_" + polutant] = nearest_station[1]
                else:
                    yearRecord[polutant] = float('nan')
                    yearRecord["Dist_" + polutant] = float('nan')
            locationMergeDataset.append(yearRecord)
                
reporting_facilities.apply(lambda x: extractNearestStation(x), axis=1)
facilities_nearby_stations = pd.DataFrame(locationMergeDataset)
facilities_nearby_stations.to_csv("E:/Research/Processing Datasets/Facility_Nearby_Station_Dataset.csv", index = False)
print("Completed Facility-Staion merge in ", time.process_time() - start)
